In [1]:
PATH = r"C:\Users\garyhu\Desktop\youtube_video_sample_audio_piano_c_major_scale.wav"
#'C:\Users\garyhu\Desktop\好聽拷貝.wav'
! pip install -U kaleido

# Configuration
FPS = 30
FFT_WINDOW_SECONDS = 0.25 # how many seconds of audio make up an FFT window

# Note range to display
FREQ_MIN = 10
FREQ_MAX = 1000

# Notes to display
TOP_NOTES = 3

# Names of the notes
NOTE_NAMES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

# Output size. Generally use SCALE for higher res, unless you need a non-standard aspect ratio.
RESOLUTION = (1920, 1080)
SCALE = 2 # 0.5=QHD(960x540), 1=HD(1920x1080), 2=4K(3840x2160)


In [2]:
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.io import wavfile  # 获取API
import os
import numpy as np

# 从GDrive获取WAV文件，例如：
# AUDIO_FILE = os.path.join(PATH,'short_popcorn.wav')

# 或者下载我的示例音频
#!wget https://github.com/jeffheaton/present/raw/master/youtube/video/sample_audio/piano_c_major_scale.wav
AUDIO_FILE = r"C:\Users\garyhu\Desktop\youtube_video_sample_audio_piano_c_major_scale.wav"

from IPython.display import Audio
Audio(AUDIO_FILE)

fs, data = wavfile.read(os.path.join(PATH, AUDIO_FILE))  # 加载数据
audio = data.T[0]  # 这是一个双声道音轨，获取第一个音轨
FRAME_STEP = (fs / FPS)  # 每个视频帧的音频样本数
FFT_WINDOW_SIZE = int(fs * FFT_WINDOW_SECONDS)
arr = np.array(audio, dtype=np.int16)
AUDIO_LENGTH = arr.size / fs


C:\Users\garyhu\AppData\Local\Temp\ipykernel_20212\828928473.py:17: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, data = wavfile.read(os.path.join(PATH, AUDIO_FILE))  # 加载数据


In [7]:
import plotly.graph_objects as go

def plot_fft(p, xf, fs, notes, dimensions=(960,540)):   
    """
    繪製FFT（快速傅立葉變換）頻譜圖，並在圖上標註頂部音符。

    參數:
    p (numpy.ndarray): 幅值頻譜。
    xf (numpy.ndarray): 頻率值。
    fs (float): 取樣頻率。
    notes (list): 要在圖上標註的頂部音符列表。
    dimensions (tuple): 圖的尺寸（寬度，高度）。

    返回:
    go.Figure: Plotly 圖形對象。
    """

    layout = go.Layout(               # 圖形參數
        title="頻譜",
        autosize=False,
        width=dimensions[0],        #長寬
        height=dimensions[1],
        xaxis_title="頻率（音符）",     #xy名稱
        yaxis_title="幅值",
        font={'size' : 24},
    )

    fig = go.Figure(layout=layout,    # go.Figure圖形形狀創建  layout設置圖形的布局參數
                layout_xaxis_range=[FREQ_MIN,FREQ_MAX], #設圖形中xy軸範圍
                layout_yaxis_range=[0,1]
                )

    fig.add_trace(go.Scatter(       #這條折線圖代表了 FFT 後得到的頻譜數據  fig.add_trace加入圖形，go.Scatter繪製折線圖
        x = xf,                 #xy參數
        y = p))
    """
    這行程式碼將一條折線圖添加到 Plotly 圖形中。
    x 軸的數據是 
    xf，即頻率值；
    y 軸的數據是 
    p，即幅值頻譜。
    """
    for note in notes:              #在 Plotly 圖形中標註頂部音符的位置和內容
        fig.add_annotation(x=note[0]+10, y=note[2],
                text=note[1],
                font = {'size' : 48},
                showarrow=False)
        """
        x=note[0]+10 表示標註的 x 軸位置，即音符的頻率值加上一個偏移值 10。
        y=note[2] 表示標註的 y 軸位置，即音符的幅值。
        text=note[1] 是標註的文本內容，即音符的名稱。
        font={'size': 48} 設置了標註的字體大小為 48。
        showarrow=False 表示不顯示箭頭。
        """
    return fig

def extract_sample(audio, frame_number):    #audio音頻數據，frame_number需要分析的偵數
    """
    提取給定幀數的音頻樣本。

    參數:
    audio (numpy.ndarray): 音頻數據。
    frame_number (int): 幀數。

    返回:
    numpy.ndarray: 提取的音頻樣本。
    """
    FRAME_OFFSET = int(len(audio) / FRAME_COUNT)
    end = frame_number * FRAME_OFFSET       #frame_number偵數，FRAME_OFFSET每個音頻樣本之間的間隔或偏移量

    begin = int(end - FFT_WINDOW_SIZE)      #提取的音頻樣本將從索引 end-FFT_WINDOW_SIVE 的值開始，( int無條件捨去小數 )


    if end == 0:
        # 沒有音頻數據，返回全零（開始）
        return np.zeros((np.abs(begin)),dtype=float)    
    elif begin<0:
        # 有一些音頻數據，填充零
        return np.concatenate([np.zeros((np.abs(begin)),dtype=float),audio[0:end]]) 
    else:
        # 通常情況，返回下一個樣本
        return audio[begin:end]             #如果沒有音頻數據則無法進行接下來步驟

def find_top_notes(fft,num):
    """
    從FFT頻譜中查找頂部音符。

    參數:
    fft (numpy.ndarray): FFT頻譜。
    num (int): 要查找的頂部音符數量。

    返回:
    list: 頂部音符列表。
    """

    if np.max(fft.real)<0.001:      #如果 FFT 頻譜中的最大實部值小於 0.001，則返回空列表
        return []                   #如果大於0.001，則按照fft.real實部值排列大到小

    lst = [x for x in enumerate(fft.real)]      #將排好的序列轉換為一個列表
    lst = sorted(lst, key=lambda x: x[1],reverse=True)  #用sorted將lst[]按照指定的鍵key，按照每個元素的第二個值為依據進行排列

    idx = 0
    found = []              #創建空列表[]
    found_note = set()      #創建空集合{}   儲存已經找到的音符
    while( (idx<len(lst)) and (len(found)<num) ):   #重複直到找到全部音符或全部跑完len<0
        f = xf[lst[idx][0]]     #f頻率
        y = lst[idx][1]            #FFT 頻譜中對應的幅值
        n = freq_to_number(f)      #計算頻率對應的音符數字
        n0 = int(round(n))         #round四捨五入
        name = note_name(n0)        #將數字轉為音符名稱

        if name not in found_note:      #加入音符到集合
            found_note.add(name)
            s = [f,note_name(n0),y]
            found.append(s)
        idx += 1
        
    return found        #返回全部音符的種類數量


In [8]:
import numpy as np
import tqdm

# 在這個範例中，註解的翻譯將使用簡體中文，您可以根據需要進行調整。

# 刪除之前製作的圖像文件
!rm /content/*.png

# 計算音符的數值表示，將任意頻率的音符轉換為一個數值表示(頻率轉數值)
def freq_to_number(f): return 69 + 12*np.log2(f/440.0)  #f為音符的頻率，440為基準點平均線 np.log2計算頻率相對於 A4 音符的倍數關係，然後取以 2 為底的對數，這樣可以得到相應的半音數量
# 計算音符對應的頻率，將音符的數值表示轉換為對應的頻率值(數值轉對應頻率值)
def number_to_freq(n): return 440 * 2.0**((n-69)/12.0)  #n代表著音符的數值表示，n-69：將音符數值的基準點調整為 A4 音符的數值表示，12.0計算半音數量，440倍數關係
# 獲取音符名稱
def note_name(n): return NOTE_NAMES[n % 12] + str(int(n/12 - 1)) #轉完對應的頻率值後就可以對應到音名，畫圖表(n % 12計算除後餘數)

# 定義漢寧窗口函數
window = 0.5 * (1 - np.cos(np.linspace(0, 2*np.pi, FFT_WINDOW_SIZE, False)))
"""
這行程式碼是用來產生漢寧窗（Hanning window）的。
漢寧窗是一種常用的窗函數，通常用於在進行傅立葉變換之前對信號進行加權，
以減少傅立葉分析中的譜漏（spectral leakage）和側葉瓣（side lobes）的影響。
"""

# 使用 np.fft.rfftfreq 函數計算 FFT 頻率
xf = np.fft.rfftfreq(FFT_WINDOW_SIZE, 1/fs)
"""這行程式碼是用來產生傅立葉變換後的頻率軸（frequency axis）。
在傅立葉變換中，通常使用的是離散傅立葉變換（DFT）或快速傅立葉變換（FFT）。
這兩種變換通常返回實部和虛部，因此對於實數信號，通常只需考慮其中一半的頻譜，即正頻譜。
"""

# 計算動畫中的幀數
FRAME_COUNT = int(AUDIO_LENGTH * FPS) #計算動畫偵數
#if FRAME_COUNT == 0:
#    raise ValueError("FRAME_COUNT 不能為零")


# 第一遍遍歷，查找最大幅值以便後續縮放
mx = 0
for frame_number in range(FRAME_COUNT):
    # 提取音頻樣本
    sample = extract_sample(audio, frame_number)    #frame_number音頻數據的幀數或時間點。
    # 使用漢寧窗口函數進行加權
    fft = np.fft.rfft(sample * window)
    # 使用快速傅立葉變換（FFT）計算加權後的音頻樣本的頻譜
    fft = np.abs(fft).real 
    # 更新最大幅值到mx
    mx = max(np.max(fft),mx)

print(f"最大幅值: {mx}")

# 第二遍遍歷，生成動畫
for frame_number in tqdm.tqdm(range(FRAME_COUNT)):
    # 提取音頻樣本
    sample = extract_sample(audio, frame_number)
    # 使用漢寧窗口函數進行加權
    fft = np.fft.rfft(sample * window)
    # 使用快速傅立葉變換（FFT）計算加權後的音頻樣本的頻譜
    fft = np.abs(fft) / mx 
    # 查找頂部音符
    s = find_top_notes(fft,TOP_NOTES)   #find_top_notes從 FFT 頻譜中查找頂部音符，即具有最高幅值的音符
    # 繪製 FFT 圖譜並標註頂部音符
    fig = plot_fft(fft.real, xf, fs, s, RESOLUTION) #plot_fft繪製 FFT 圖譜並在圖上標註頂部音符
    # 將圖像保存為圖片文件
    fig.write_image(f"/content/frame{frame_number}.png",scale=2)    #以便之後串成動畫  scale=2將圖片放大2倍存檔，增加解析度

"""
這段程式碼是用來製作一個音頻頻譜的動畫。
首先，它會遍歷音頻文件的每個小部分（稱為幀），
並計算每個幀的 FFT（快速傅立葉變換），這樣可以將幅值頻譜表示為頻率的函數。
然後，它找到了所有幀中的最大幅值，以便將所有幀的頻譜進行縮放，使其適合於動畫中。
接著，它再次遍歷每個幀，將 FFT 的結果轉換為頻譜圖，並在圖上標註出一些頂部的音符，
最後將每個幀的圖像保存為一個圖像文件，以便後續的動畫製作。
"""

'rm' is not recognized as an internal or external command,
operable program or batch file.


最大幅值: 1035.1945494237968


  0%|          | 0/418 [00:00<?, ?it/s]